In [4]:
import os, io
import pandas as pd
from pathlib import Path
from deep_parser import TextFromFile, TextFromWeb
from deep_parser.helpers.errors import DocumentProcessingError
from tqdm.auto import tqdm
from glob import glob
import base64, json
import timeout_decorator
import pandas as pd
import numpy as np
import json

In [ ]:
#df_name = "all_hum_data_24-12-2022.csv.gz"
#leads_df = pd.read_csv(
#    df_name,
#    compression="gzip",
#    low_memory=False,
#    lineterminator="\n",
#    usecols=["lead_id", "project_id", "url"],
#).drop_duplicates()

In [ ]:
leads_df.to_csv('projects_leads_urls.csv', index=None)

In [ ]:
output_path = 'pulled_leads'
output_path = Path(output_path)
output_path.mkdir(exist_ok=True, parents=True)

non_url_input_path = Path('pdfs-total')

In [ ]:
def pull_websites():
    print("###################### Start puling websites data.")
    
    for (project_id, lead_id), group in tqdm(
        list(leads_df.groupby(["project_id", "lead_id"]))
    ):
        project_id, lead_id = str(int(project_id)), str(int(lead_id))
        urls = group["url"].unique()

        assert len(urls) == 1

        url = urls[0]

        if not isinstance(url, str):  # possibly np.nan
            continue

        if url.endswith(".pdf"):
            print(f"{url} is a PDF!")
            try:
                parser = TextFromFile(url=url, from_web=True)
                text, _ = parser.extract_text(output_format="list")
            except (RuntimeError, DocumentProcessingError, Exception) as e:
                print(f"Error {e} on PDF url {url}")
                continue
        else:
            try:
                parser = TextFromWeb(url=url)
                text = parser.extract_text(output_format="list")
                parser.close()
            except (RuntimeError, DocumentProcessingError, Exception) as e:
                print(f"Error {e} on standard url {url}")
                continue

        nested_dir_path = output_path / project_id
        nested_dir_path.mkdir(exist_ok=True, parents=True)

        file_path = nested_dir_path / f"{lead_id}.json"
        if file_path.exists():
            continue

        text = None

        with open(file_path, "w+") as f:
            json.dump(text, f)

In [ ]:
@timeout_decorator.timeout(5 * 60, use_signals=False)
def extract(in_path, out_path):
    if out_path.exists():
        return False

    with open(in_path, "rb") as f:
        binary = base64.b64encode(f.read())

    try:
        document = TextFromFile(stream=binary, ext="pdf")
        text, _ = document.extract_text(output_format="list")
    except (RuntimeError, DocumentProcessingError):
        return False

    try:
        # open(out_path, "w").write(text)
        with open(out_path, "w+") as f:
            json.dump(text, f)
    except UnicodeEncodeError:
        return False

    return True


def work(in_path, out_path):
    try:
        return extract(in_path, out_path)
    except timeout_decorator.TimeoutError:
        print(f"Timeout for File {in_path}!")


def pull_pdfs():
    print("###################### Start pulling pdf data.")

    name_to_id = {
        row["url"].rstrip("/").split("/")[-1]: (int(row["lead_id"]), int(row["project_id"]))
        for _, row in leads_df.iterrows()
        if row["url"] is not np.nan
    }

    paths = [Path(p) for p in glob(str(non_url_input_path / "*.pdf"))]

    for in_path in tqdm(paths):
        name = Path(in_path).name
        lead_id, project_id = name_to_id[name]

        out_path = output_path / str(project_id) / f"{lead_id}.json"
        out_path.parent.mkdir(exist_ok=True, parents=True)

        work(in_path, out_path)

In [ ]:
# repull data
# pull urls
# pull pdf
# solve duplicates issue
# run code